In [15]:
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer
from FlagEmbedding import BGEM3FlagModel
import numpy as np
import torch

model_name = "BAAI/bge-m3"
a = "这是个阴谋"
b = "This is a test sentence"

In [18]:
def mean_pooling(embedings):
    return embedings.mean(axis=0)

def noramlize(x):
    return x / np.linalg.norm(x, axis=0)


def get_sentencen_trans_embed(text_input, cls=False):
    with torch.no_grad():
        sentence_model = SentenceTransformer(model_name, trust_remote_code=True)

    token_embs = sentence_model.encode(text_input, output_value="token_embeddings", normalize_embeddings=True)
    if cls:
        return token_embs.detach().cpu().numpy()[0]
    return token_embs.detach().cpu().numpy()[1:]


def get_flag_embed(text_input, cls=False):
    model = BGEM3FlagModel(model_name,  use_fp16=True) 


    output_1 = model.encode([text_input], return_dense=True, return_sparse=True, return_colbert_vecs=True)

    if cls:
        return output_1['dense_vecs']
    return output_1['colbert_vecs']

In [20]:
sentence_embed_a = noramlize(mean_pooling(get_sentencen_trans_embed(a)))
sentence_embed_b = noramlize(mean_pooling(get_sentencen_trans_embed(b)))

print(np.dot(sentence_embed_a, sentence_embed_b))

flag_embed_a = noramlize(mean_pooling(get_flag_embed(a)))
flag_embed_b = noramlize(mean_pooling(get_flag_embed(b)))
print(np.dot(flag_embed_a, flag_embed_b))


# print(sentence_embed)
# print(flag_embed)

# print(sentence_embed == flag_embed)

0.72376466


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 91312.86it/s]


AttributeError: 'list' object has no attribute 'mean'

In [11]:
sentence_embed_a = noramlize(get_sentencen_trans_embed(a, cls=True))
sentence_embed_b = noramlize(get_sentencen_trans_embed(b, cls=True))

print(np.dot(sentence_embed_a, sentence_embed_b))

flag_embed_a = get_flag_embed(a, cls=True)
flag_embed_b = get_flag_embed(b, cls=True)
print(np.dot(flag_embed_a, flag_embed_b))

0.5701115


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 271183.45it/s]


0.5703


In [ ]:
from chunked_pooling.wrappers import BAAIBGEM3Wrapper
